In [ ]:
%load_ext autoreload
%autoreload 2
from src.VLLMServer import launch_command,get_client_dict,ask_llm,ask_llm_prompt

In [ ]:
import yaml

In [ ]:
conf_path="config.yaml"

with open(conf_path,"r") as f:
    conf=yaml.safe_load(f.read())
client_dict=get_client_dict(conf)
#print(launch_command(conf))

client_dict

In [ ]:
model_name="hatakeyama-llm-team/Tanuki-8B-Instruct"
model_name="microsoft/Phi-3-medium-128k-instruct"
#model_name="Rakuten/RakutenAI-7B-chat"
#model_name="karakuri-ai/karakuri-lm-8x7b-chat-v0.1"
#model_name="Qwen/Qwen2-7B-Instruct"
#model_name="models--mistralai--Mixtral-8x22B-Instruct-v0.1"
model_name="llm-jp/llm-jp-13b-instruct-full-ac_001_16x-dolly-ichikara_004_001_single-oasst-oasst2-v2.0"
model_name="nvidia/Nemotron-4-340B-Instruct"

question="純粋理性批判はたぬきに理解できますか?"
#question="以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。### 指示:\n"
res=ask_llm_prompt(client_dict,model_name,question)
res

In [ ]:
prompt="以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。### 指示:\n元気ですか\n\n### 応答:\n"
client = client_dict[model_name]["client"]
completion = client.completions.create(model=model_name,
                                        prompt=prompt,
                                        max_tokens=int(client_dict[model_name]["config"]["max_tokens"]))
out=completion.choices[0].text.strip()  # .message.content.strip()
out


In [ ]:
cmd="""以下のテキストをもとに､日本語で教科書風の文章を生成しなさい｡
-[問題文]を出力
-[考え方]を出力
-[答え]を出力
-[詳細な解説]を出力
"""
#cmd="以下のテキストをもとに､複数の登場人物でなされる日本語の会話文を生成しなさい｡"
question=cmd+"""
#テキスト
Q.
Rick took off on a road trip for the summer. He traveled to the first destination on his trip, and then from there, he traveled twice the distance to this second destination. The third point on his trip was 40 miles away, which was half the distance he traveled from the beginning to his first destination. The final destination took twice the amount of driving of all the other legs put together. How many miles in total did he travel across all the legs of the trip?

A.	
Let's solve this problem using Python's sympy library. <llm-code> import sympy as sp # third leg was 40 miles x = 40 # third leg was half the first leg first_leg = 2 * x # second leg was twice the first leg second_leg = 2 * first_leg # fourth leg was equal to the sum of the three legs fourth_leg = x + first_leg + second_leg # adding up the four to get the total distance total_distance = x + first_leg + second_leg + fourth_leg total_distance </llm-code> <llm-code-output> 560 </llm-code-output> Thus Rick traveled \boxed{560} miles in total.

"""
res=ask_llm_prompt(client_dict,model_name,question)
print(res)

In [ ]:
model_name

In [ ]:
client = client_dict[model_name]["client"]

template1 = client_dict[model_name]["config"]["template1"]
template2 = client_dict[model_name]["config"]["template2"]
prompt = template1+question+template2
completion = client.completions.create(model=model_name,
                                        prompt=prompt,
                                        max_tokens=int(client_dict[model_name]["config"]["max_tokens"]))


In [ ]:

from datasets import load_dataset
ds=load_dataset("tokyotech-llm/Swallow-Instruct-v0.1",split="train")
ds

In [ ]:
q_list=[]

for record in ds:
    for conv in record["conversation"]:
        if conv["role"]=="user":
            q_list.append(conv["content"])

In [ ]:
import pandas as pd
df=pd.DataFrame(q_list)
df.to_csv("swallow.csv")